In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle

image_dict = pickle.load( open( "image_dict.pickle", "rb" ) )
one_hot_score_dict =  pickle.load( open( "one_hot_score_dict.pickle", "rb" ) )


In [2]:
#print len(image_dict) #we have 16913 images
#get list of unique keys
list_of_keys = image_dict.keys()
#select a random 2,913 indices
random_shuffle_keys = list_of_keys
from random import shuffle
shuffle(random_shuffle_keys)
#print random_shuffle_keys[0:10]
#split the data of both the image arrays, and the one_hots
#get the last 2913 keys
test_keys = random_shuffle_keys[-2913:]
train_keys = random_shuffle_keys[0:-2913]
#print len(test_keys)
#print len(train_keys)

config = tf.ConfigProto()
config.gpu_options.allocator_type = 'BFC'


In [3]:
n_classes = 11
batch_size = 128

x = tf.placeholder('float', [None, 76800])
y = tf.placeholder('float')

keep_rate = 0.8
keep_prob = tf.placeholder(tf.float32)


In [4]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

def maxpool2d(x):
    #                        size of window         movement of window
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,4,4,1], padding='SAME')



In [5]:
def convolutional_neural_network(x):
    weights = {'W_conv1':tf.Variable(tf.random_normal([5,5,3,32])),
               'W_conv2':tf.Variable(tf.random_normal([5,5,32,64])),
               'W_fc':tf.Variable(tf.random_normal([10*10*64,1024])),
               'out':tf.Variable(tf.random_normal([1024, n_classes]))}

    biases = {'b_conv1':tf.Variable(tf.random_normal([32])),
               'b_conv2':tf.Variable(tf.random_normal([64])),
               'b_fc':tf.Variable(tf.random_normal([1024])),
               'out':tf.Variable(tf.random_normal([n_classes]))}

    x = tf.reshape(x, shape=[-1, 160, 160, 3])

    conv1 = tf.nn.relu(conv2d(x, weights['W_conv1']) + biases['b_conv1'])
    conv1 = maxpool2d(conv1)
    
    conv2 = tf.nn.relu(conv2d(conv1, weights['W_conv2']) + biases['b_conv2'])
    conv2 = maxpool2d(conv2)

    fc = tf.reshape(conv2,[-1, 10*10*64])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
    fc = tf.nn.dropout(fc, keep_rate)

    output = tf.matmul(fc, weights['out'])+biases['out']

    return output
    

In [34]:
def train_neural_network(x):
    prediction = convolutional_neural_network(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(prediction,y) )
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    hm_epochs = 10
    with tf.Session(config = config) as sess:
        sess.run(tf.global_variables_initializer())

        for epoch in range(hm_epochs):
            epoch_loss = 0
            for i in range(int(len(train_keys)/batch_size)):
                keys_for_batch = train_keys[(i*batch_size):(i+1)*batch_size]
                epoch_x = np.array([image_dict[value].flatten() for value in keys_for_batch])
                epoch_y = np.array([one_hot_score_dict[value] for value in keys_for_batch])
                #print 'Size of batch: ' + str(len(epoch_x))
                i, c = sess.run([optimizer, cost], feed_dict = {x: epoch_x, y: epoch_y})
                epoch_loss += c

            print 'Epoch ' + str(epoch+1) + ' completed out of ' + str(hm_epochs), ' loss:' + str(epoch_loss)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        pred_label = tf.argmax(prediction, 1)
        corr_convert = tf.argmax(y,1)
        #print 'Accuracy: ' + str(accuracy.eval({x:np.array([image_dict[value].flatten() for value in test_keys[:1000]])
        #                                                  , y:np.array([one_hot_score_dict[value] for value in test_keys[:1000]
        #                                                               ])}))
        num_correct = 0
        for i in range(len(test_keys)):
            value = test_keys[i]
            val = accuracy.eval({x:np.array([image_dict[value].flatten()])
                                                          , y:np.array([one_hot_score_dict[value]
                                                                       ])})
            if val == 1 or val == 1.0:
                num_correct += 1
            eval_list[i] = val
            #feed_dict = {x: np.array([image_dict[value].flatten()])}
            classification = pred_label.eval( {x: np.array([image_dict[value].flatten()])})
            class_list[i] = classification[0]
            correct_list[i] = corr_convert.eval({y:np.array([one_hot_score_dict[value]])})[0]
    print 'Num correct: ' + str(num_correct) 
    print 'Completed'

In [35]:
#get lists we need to append to, run the network
eval_list = [None]*2913
class_list = {}
correct_list = {}
train_neural_network(x)

Epoch 1 completed out of 10  loss:285299124.688
Epoch 2 completed out of 10  loss:39236049.4004
Epoch 3 completed out of 10  loss:62184.3274343
Epoch 4 completed out of 10  loss:245.089285851
Epoch 5 completed out of 10  loss:239.472841501
Epoch 6 completed out of 10  loss:234.561125994
Epoch 7 completed out of 10  loss:230.282131314
Epoch 8 completed out of 10  loss:226.57831192
Epoch 9 completed out of 10  loss:223.395252585
Epoch 10 completed out of 10  loss:220.678839207
Num correct: 599
Completed


In [36]:
len(test_keys)

2913

In [5]:
#Accuracy
print 599*1.0/2913

0.205629934775


In [38]:
#eval_list = [None]*2913

In [8]:
#debugging, ignore
feed_dict = {x: [your_image]}
classification = tf.run(y, feed_dict)


[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

In [31]:
print class_list[0]

4


In [36]:
#store the result
with open('cnn_corr_list.pickle', 'wb') as f:
    pickle.dump(correct_list, f)

In [37]:
with open('cnn_pred_list.pickle', 'wb') as f:
    pickle.dump(class_list, f)

In [39]:
#examine lift
correct_list

{0: 5,
 1: 8,
 2: 8,
 3: 9,
 4: 8,
 5: 5,
 6: 9,
 7: 6,
 8: 4,
 9: 6,
 10: 5,
 11: 8,
 12: 7,
 13: 9,
 14: 8,
 15: 9,
 16: 10,
 17: 3,
 18: 8,
 19: 9,
 20: 4,
 21: 6,
 22: 8,
 23: 8,
 24: 8,
 25: 3,
 26: 8,
 27: 6,
 28: 9,
 29: 8,
 30: 6,
 31: 8,
 32: 8,
 33: 9,
 34: 9,
 35: 3,
 36: 6,
 37: 9,
 38: 6,
 39: 9,
 40: 5,
 41: 6,
 42: 7,
 43: 6,
 44: 7,
 45: 8,
 46: 9,
 47: 5,
 48: 7,
 49: 5,
 50: 8,
 51: 8,
 52: 8,
 53: 7,
 54: 8,
 55: 6,
 56: 6,
 57: 9,
 58: 9,
 59: 8,
 60: 8,
 61: 7,
 62: 6,
 63: 7,
 64: 8,
 65: 4,
 66: 3,
 67: 4,
 68: 8,
 69: 9,
 70: 7,
 71: 9,
 72: 9,
 73: 8,
 74: 7,
 75: 8,
 76: 8,
 77: 5,
 78: 6,
 79: 5,
 80: 8,
 81: 6,
 82: 9,
 83: 7,
 84: 8,
 85: 4,
 86: 8,
 87: 9,
 88: 7,
 89: 4,
 90: 3,
 91: 5,
 92: 6,
 93: 7,
 94: 8,
 95: 5,
 96: 6,
 97: 6,
 98: 7,
 99: 6,
 100: 6,
 101: 8,
 102: 9,
 103: 8,
 104: 4,
 105: 8,
 106: 3,
 107: 10,
 108: 8,
 109: 9,
 110: 7,
 111: 9,
 112: 7,
 113: 7,
 114: 8,
 115: 8,
 116: 8,
 117: 9,
 118: 9,
 119: 3,
 120: 1,
 121: 7,
 122: 10,


In [40]:
#get some of squares
sum_of_squares = 0
for i in range(len(test_keys)):
    sum_of_squares += (correct_list[i] - class_list[i])**2
print sum_of_squares

8657
